# MovieLens

In [2]:
import numpy as np
import pandas as pd

train_data = np.load('./RippleNet-PyTorch-baseline/data/train_data_original.npy', allow_pickle=True)
test_data = np.load('./RippleNet-PyTorch-baseline/data/test_data_original.npy', allow_pickle=True)
eval_data = np.load('./RippleNet-PyTorch-baseline/data/eval_data_original.npy', allow_pickle=True)
# 把train和eval合并成train，test单独test，过滤掉第三列是0的
# 合并train和eval数据
train_data = np.concatenate([train_data, eval_data])

# 过滤掉第三列(rating)为0的数据
train_data = train_data[train_data[:, 2] != 0]
test_data = test_data[test_data[:, 2] != 0]
# 变成dataframe
train_data = pd.DataFrame(train_data, columns=['user', 'item', 'interaction'])
test_data = pd.DataFrame(test_data, columns=['user', 'item', 'interaction'])
ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
# 通过train data和test data，和ratingsmerge
train_data = pd.merge(train_data, ratings, left_on=['user', 'item'], right_on=['userId', 'movieId'], how='left')
train_data = train_data.drop(['user', 'item','interaction'], axis=1)
test_data = pd.merge(test_data, ratings, left_on=['user', 'item'], right_on=['userId', 'movieId'], how='left')
test_data = test_data.drop(['user', 'item','interaction'], axis=1)



/var/folders/4x/xyn0bgpj7sv8g79hf_w6b6vm0000gn/T/ipykernel_77241/509847103.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])


In [3]:
train_data

,userId,movieId,rating,timestamp
0,1,2355,5,978824291
1,1,2797,4,978302039
2,1,1097,4,978301953
3,1,2804,5,978300719
4,1,260,4,978300760
...,...,...,...,...
301850,3704,562,5,966284788
301851,4892,300,4,962735284
301852,2585,260,4,973876019
301853,2507,3949,5,988075138


In [4]:
train_data.groupby('userId').count().describe()

,movieId,rating,timestamp
count,6030.000000,6030.000000,6030.000000
mean,50.058872,50.058872,50.058872
std,54.247227,54.247227,54.247227
min,1.000000,1.000000,1.000000
25%,15.000000,15.000000,15.000000
50%,31.000000,31.000000,31.000000
75%,65.000000,65.000000,65.000000
max,729.000000,729.000000,729.000000


In [5]:
test_data.groupby('userId').count().describe()


,movieId,rating,timestamp
count,5876.000000,5876.000000,5876.000000
mean,12.765487,12.765487,12.765487
std,13.949214,13.949214,13.949214
min,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000
50%,8.000000,8.000000,8.000000
75%,17.000000,17.000000,17.000000
max,171.000000,171.000000,171.000000


In [6]:
train_data.to_csv('dataset/movielens_1M/train_set.csv', index=False)
test_data.to_csv('dataset/movielens_1M/test_set.csv', index=False)


In [7]:
import pickle
train_data = pd.concat([train_data, test_data])

# 保存用户ID列表
with open('dataset/movielens_1M/user_ids.pkl', 'wb') as f:
    pickle.dump(train_data['userId'].unique().tolist(), f)

# 保存电影ID列表 
with open('dataset/movielens_1M/movie_ids.pkl', 'wb') as f:
    pickle.dump(train_data['movieId'].unique().tolist(), f)

-----
DPO Dataset

In [8]:
import pandas as pd
ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
links = pd.read_csv('dataset/movielens/links.csv')
links.dropna(inplace=True)
movies_metadata = pd.read_csv('dataset/movielens/movies_metadata.csv')

/var/folders/4x/xyn0bgpj7sv8g79hf_w6b6vm0000gn/T/ipykernel_77241/432755173.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
/var/folders/4x/xyn0bgpj7sv8g79hf_w6b6vm0000gn/T/ipykernel_77241/432755173.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('dataset/movielens/movies_metadata.csv')


link：

movieId(=movieId in ratings_small),

imdbId (imdb_id in movies_metadata), 

tmdbId (id in movies_metadata)

In [9]:
movies_metadata['id'] = movies_metadata['id'].astype('str')
links['tmdbId'] = links['tmdbId'].astype(int).astype('str')
movie = pd.merge(
    links[['movieId', 'tmdbId']],
    movies_metadata[['id', 'title', 'genres','original_language','popularity','overview']],
    left_on='tmdbId',
    right_on='id',
    how='left'
)

In [10]:
def parse_genres(x):
    if pd.isna(x):
        return []
    try:
        if isinstance(x, str):
            return eval(x)
        return x
    except:
        return []
movie['genres'] = movie['genres'].apply(parse_genres)
movie['genres'] = movie['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) and x and isinstance(x[0], dict) else x)


In [11]:
movie.head()

,movieId,tmdbId,id,title,genres,original_language,popularity,overview
0,1,862,862,Toy Story,"[Animation, Comedy, Family]",en,21.946943,"Led by Woody, Andy's toys live happily in his ..."
1,2,8844,8844,Jumanji,"[Adventure, Fantasy, Family]",en,17.015539,When siblings Judy and Peter discover an encha...
2,3,15602,15602,Grumpier Old Men,"[Romance, Comedy]",en,11.7129,A family wedding reignites the ancient feud be...
3,4,31357,31357,Waiting to Exhale,"[Comedy, Drama, Romance]",en,3.859495,"Cheated on, mistreated and stepped on, the wom..."
4,5,11862,11862,Father of the Bride Part II,[Comedy],en,8.387519,Just when George Banks has recovered from his ...


In [12]:
a = [len(movie["overview"][i]) for i in range(40000) if isinstance(movie["overview"][i], str)]
average = sum(a) / len(a) if len(a) > 0 else 0
average

324.4713833103378

In [13]:
test_data.groupby('userId').count().describe()


,movieId,rating,timestamp
count,5876.000000,5876.000000,5876.000000
mean,12.765487,12.765487,12.765487
std,13.949214,13.949214,13.949214
min,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000
50%,8.000000,8.000000,8.000000
75%,17.000000,17.000000,17.000000
max,171.000000,171.000000,171.000000


In [14]:
seq_dataset = pd.read_csv('dataset/movielens_1M/sequential_top10.csv')
seq_dataset

,userId,history,candidates,groundtruth
0,1,"[1270, 1836, 3408, 2028, 150, 1028, 1246, 2762...","[1453, 3105, 1431, 2791, 198, 2568, 2507]","[2791, 3105]"
1,1,"[1836, 3408, 260, 2692, 1961, 1028, 1097, 2918...","[2530, 3105, 1379, 2511, 1063, 2791, 1272]","[2791, 3105]"
2,1,"[3186, 2804, 2692, 150, 1028, 2797, 2918, 3114...","[3563, 3105, 1058, 3478, 3533, 631, 2791, 356]","[2791, 3105]"
3,1,"[3186, 1836, 3408, 2797, 2762, 3114, 1545, 527...","[590, 1921, 745, 1280, 3210, 2312, 3105, 1005,...","[2791, 3105]"
4,1,"[3408, 2804, 260, 2692, 1961, 1028, 1097, 2797...","[3611, 2290, 1064, 3749, 3837, 2449, 823, 3105...","[2791, 3105]"
...,...,...,...,...
154225,6040,"[1096, 2070, 1250, 296, 80, 899, 1206, 1265, 3...","[2437, 3262, 3610, 1960, 3896, 673, 3032, 1333...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154226,6040,"[1392, 3067, 908, 1172, 1354, 1265, 2395, 3751...","[1002, 746, 396, 1080, 70, 1235, 369, 1191, 39...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154227,6040,"[1961, 527, 1288, 908, 1247, 2938, 1179, 3083,...","[1095, 514, 714, 2424, 3776, 3612, 3262, 1333,...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154228,6040,"[1097, 2076, 3224, 1247, 899, 1103, 2303, 3547...","[926, 720, 1207, 126, 280, 1228, 1632, 647, 12...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."


In [15]:
# 筛选掉groundtruth长度<=10的
seq_dataset = seq_dataset[seq_dataset['groundtruth'].apply(lambda x: len(eval(x)) >=10)]
seq_dataset = seq_dataset.reset_index(drop=True)

In [16]:
seq_dataset.to_csv('dataset/movielens_1M/sequential_top10_filtered.csv', index=False)

In [17]:
len(seq_dataset)

76020

In [18]:
# Combine all the movie IDs from history, candidates, and groundtruth
all_movies = set()
data=seq_dataset.copy()
data['history'] = data['history'].apply(lambda x: eval(x) if isinstance(x, str) else x)
data['candidates'] = data['candidates'].apply(lambda x: eval(x) if isinstance(x, str) else x)
data['groundtruth'] = data['groundtruth'].apply(lambda x: eval(x) if isinstance(x, str) else x)

for _, row in data.iterrows():
    all_movies.update(row['history'])
    all_movies.update(row['candidates'])
    all_movies.update(row['groundtruth'])
    #print(all_movies)

# Calculate the total number of unique movies
unique_movie_count = len(all_movies)
print(f'Total number of unique movies: {unique_movie_count}')

Total number of unique movies: 2254


In [19]:
import pandas as pd

# 加载测试集数据
test_set_path = '/Users/lavander/cpan/h411711/dataset/movielens_1M/test_set.csv'
test_set = pd.read_csv(test_set_path)

# 获取test_set中所有的movieId集合
test_movie_ids = set(test_set['movieId'].values)

# 找出不在all_movies中的movieId
movies_not_in_all_movies = test_movie_ids - all_movies

# 统计总电影数和不在all_movies的电影数
total_movies_in_test = len(test_movie_ids)
movies_not_in_all_count = len(movies_not_in_all_movies)

# 输出结果
print(f"Total number of movies in test set: {total_movies_in_test}")
print(f"Number of movies not in all_movies: {movies_not_in_all_count}")
print(f"Movies not in all_movies: {movies_not_in_all_movies}")

Total number of movies in test set: 2011
Number of movies not in all_movies: 0
Movies not in all_movies: set()


In [24]:
# import pandas as pd
# import nest_asyncio
# import asyncio
# from openai import AsyncOpenAI
# from typing import List

# # Apply nest_asyncio to fix the event loop issue
# nest_asyncio.apply()

# # AsyncOpenAI dispatch function
# async def dispatch_openai_requests(
#     messages_list: List[str],
#     model: str,
#     temperature: float,
#     max_tokens: int,
# ) -> List[str]:
#     async with AsyncOpenAI(
#         api_key="sk-KLLmOqkRpG6YnCXUwisjTw",
#         base_url="https://cmu.litellm.ai",
#     ) as client:za
#         async def single_request(text: str) -> str:
#             try:
#                 if 'gpt' in model:
#                     response = await client.chat.completions.create(
#                         model=model,
#                         messages=[
#                             {"role": "system", "content": "You are a helpful assistant."},
#                             {"role": "user", "content": text},
#                         ],
#                         temperature=temperature,
#                         max_tokens=max_tokens,
#                     )
#                     result = response.choices[0].message.content
#                 else:
#                     response = await client.completions.create(
#                         model=model,
#                         prompt=text,
#                         temperature=temperature,
#                         max_tokens=max_tokens,
#                     )
#                     result = response.choices[0].text
#                 return result.strip()
#             except Exception as e:
#                 print(f"Error in request: {str(e)}")
#                 return f"Error: {str(e)}"

#         tasks = [single_request(text) for text in messages_list]
#         return await asyncio.gather(*tasks)


In [25]:
# import nest_asyncio

# filtered_movies = movie[movie['movieId'].isin(all_movies)]

# # Step 4: Prepare overview prompts for OpenAI
# messages = [f"Summarize this overview into 10 words: {overview}" for overview in filtered_movies['overview']]

# # Step 5: Run OpenAI API requests to summarize overviews
# async def summarize_overviews(messages: List[str]):
#     model = "gpt-4o-mini"
#     temperature = 0.7
#     max_tokens = 20
#     summaries = await dispatch_openai_requests(messages, model, temperature, max_tokens)
#     return summaries

# summaries = await summarize_overviews(messages)
# # Step 6: Add the summaries as a new column
# filtered_movies['short_overview'] = summaries

# # Step 7: Save the updated dataset
# filtered_movies.to_csv("dataset/movielens_1M/used_movies.csv", index=False)
# print("Updated movie dataset saved to updated_movies.csv")


In [26]:
# # Step 1: Identify rows with errors
# errors_to_fix = filtered_movies[filtered_movies['short_overview'].str.contains("Error: <!DOCTYPE html>", na=False)]


# # Step 2: Prepare messages for rows with errors
# messages_with_errors = [
#     f"Summarize this overview into 10 words: {overview}"
#     for overview in errors_to_fix['overview']
# ]

# # Step 3: Rerun the summarization process for rows with errors
# async def summarize_error_overviews(messages: List[str]):
#     model = "gpt-4o-mini"
#     temperature = 0.7
#     max_tokens = 20
#     summaries = await dispatch_openai_requests(messages, model, temperature, max_tokens)
#     return summaries

# # Rerun summarization for errored rows
# new_summaries = await summarize_error_overviews(messages_with_errors)

# # Step 4: Update the errored rows in the DataFrame
# filtered_movies.loc[errors_to_fix.index, 'short_overview'] = new_summaries

# # Step 5: Save the updated dataset
# filtered_movies.to_csv("dataset/movielens_1M/used_movies_fixed.csv", index=False)
# print("Updated movie dataset with fixed errors saved to used_movies_fixed.csv")


In [27]:
# summaries
# filtered_movies.to_csv("dataset/movielens_1M/used_movies.csv", index=False)


In [28]:
filtered_movies=pd.read_csv("dataset/movielens_1M/used_movies.csv")

In [38]:
filtered_movies

,movieId,tmdbId,id,title,genres,original_language,popularity,overview,short_overview
0,1,862,862,Toy Story,"['Animation', 'Comedy', 'Family']",en,21.946943,"Led by Woody, Andy's toys live happily in his ...",Woody and Buzz overcome rivalry to reunite wit...
1,2,8844,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",en,17.015539,When siblings Judy and Peter discover an encha...,"Siblings play enchanted game, freeing trapped ..."
2,3,15602,15602,Grumpier Old Men,"['Romance', 'Comedy']",en,11.712900,A family wedding reignites the ancient feud be...,Wedding feud reignites between neighbors; divo...
3,4,31357,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",en,3.859495,"Cheated on, mistreated and stepped on, the wom...","Women seek better relationships, discussing ex..."
4,5,11862,11862,Father of the Bride Part II,['Comedy'],en,8.387519,Just when George Banks has recovered from his ...,George faces unexpected changes with news of t...
...,...,...,...,...,...,...,...,...,...
2239,3944,21370,21370,Bootmen,"['Comedy', 'Drama', 'Music']",en,1.400650,Sean Odkin loves to dance much to his fathers ...,Sean Odkin seeks fulfillment through dance des...
2240,3945,20455,20455,Digimon: The Movie,"['Fantasy', 'Animation', 'Science Fiction', 'F...",en,5.300501,The first story focused on Tai and Kari Kamiya...,"Tai and Kari discover Digimon, encounter adven..."
2241,3949,641,641,Requiem for a Dream,"['Crime', 'Drama']",en,6.874828,The hopes and dreams of four ambitious people ...,Four ambitious individuals' lives are ruined b...
2242,3950,10687,10687,Tigerland,"['Drama', 'War']",en,5.878885,A group of recruits go through Advanced Infant...,Recruits undergo Advanced Infantry Training at...


根据movie这个大表，和seq_dataset, 构建自然语言数据集

Natural Language dataset format: every row contains
1. sequential historical interactions (Movie ID + Movie Names + genre + spoken Language + popularDegree)
2. candidates (Movie ID + Movie Name + genre + spoken Language + popularDegree)
3. Groudtruth (Movie ID + Movie Name)


In [52]:
def get_movie_description(row):
    # Ensure genres is properly processed
    genres = row['genres']
    if isinstance(genres, str):  # If genres are stored as a string, evaluate it to a list
        genres = eval(genres)
    if not isinstance(genres, list):  # Ensure genres is a list
        genres = [str(genres)]

    return f"Movie {row['movieId']}: {row['title']} (Genres: {', '.join(genres)}; " \
           f"Language: {row['original_language']}; Overview: {row['short_overview']})"

def process_sequence(row):
    history = eval(row['history'])
    history_movies = filtered_movies[filtered_movies['movieId'].isin(history)].apply(get_movie_description, axis=1).tolist()
    
    candidates = eval(row['candidates'])
    candidate_movies = filtered_movies[filtered_movies['movieId'].isin(candidates)].apply(get_movie_description, axis=1).tolist()
    
    next_items = eval(row['groundtruth'])
    ground_truth = filtered_movies[filtered_movies['movieId'].isin(next_items)].apply(get_movie_description, axis=1).tolist()
    
    return pd.Series({
        'user_id': row['userId'],
        'history': ','.join(history_movies),
        'candidates': ','.join(candidate_movies), 
        'ground_truth': ','.join(ground_truth)
    })

In [30]:
# def get_movie_description(row):
#     """
#     Formats the movie details into a string description.
#     """
#     return {
#         'movieId': row['movieId'],
#         'description': f"Movie {row['movieId']}: {row['title']} (Genres: {', '.join(row['genres'])}; " \
#                        f"Language: {row['original_language']}; Overview: {row['short_overview']})"
#     }

# def process_sequence(row):
#     """
#     Processes each row to generate structured 'history', 'candidates', and 'ground_truth' as lists of dictionaries.
#     """
#     # Process 'history'
#     history = eval(row['history'])
#     history_movies = filtered_movies[filtered_movies['movieId'].isin(history)].apply(get_movie_description, axis=1).tolist()
    
#     # Process 'candidates'
#     candidates = eval(row['candidates'])
#     candidate_movies = filtered_movies[filtered_movies['movieId'].isin(candidates)].apply(get_movie_description, axis=1).tolist()
    
#     # Process 'ground_truth'
#     next_items = eval(row['groundtruth'])
#     ground_truth = filtered_movies[filtered_movies['movieId'].isin(next_items)].apply(get_movie_description, axis=1).tolist()
    
#     # Return the processed data
#     return pd.Series({ 
#         'user_id': row['userId'],
#         'history': history_movies,  # Store as list
#         'candidates': candidate_movies,  # Store as list
#         'ground_truth': ground_truth  # Store as list
#     })


In [53]:
natural_language_dataset = seq_dataset[:10].apply(process_sequence, axis=1)


In [56]:
seq_dataset

,userId,history,candidates,groundtruth
0,5,"[3083, 50, 176, 562, 1171, 1759, 1392, 1732, 5...","[714, 669, 1715, 3852, 509, 2311, 32, 1439, 36...","[908, 1715, 714, 1250, 1921, 1535, 1175, 1635,..."
1,5,"[2908, 2959, 2333, 896, 162, 1649, 728, 3267, ...","[792, 2765, 1715, 2461, 1, 2142, 470, 1175, 22...","[908, 1715, 714, 1250, 1921, 1535, 1175, 1635,..."
2,5,"[1046, 296, 1192, 176, 1730, 1649, 1885, 1392,...","[1921, 3875, 1715, 1639, 669, 1535, 31, 509, 1...","[908, 1715, 714, 1250, 1921, 1535, 1175, 1635,..."
3,5,"[2770, 2908, 593, 1192, 176, 1730, 562, 162, 5...","[32, 255, 1715, 784, 1250, 1058, 3601, 908, 19...","[908, 1715, 714, 1250, 1921, 1535, 1175, 1635,..."
4,5,"[2770, 2390, 3083, 2571, 562, 162, 2291, 1171,...","[3665, 1011, 352, 1381, 1175, 1535, 3816, 1722...","[908, 1715, 714, 1250, 1921, 1535, 1175, 1635,..."
...,...,...,...,...
76015,6040,"[1096, 2070, 1250, 296, 80, 899, 1206, 1265, 3...","[2437, 3262, 3610, 1960, 3896, 673, 3032, 1333...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
76016,6040,"[1392, 3067, 908, 1172, 1354, 1265, 2395, 3751...","[1002, 746, 396, 1080, 70, 1235, 369, 1191, 39...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
76017,6040,"[1961, 527, 1288, 908, 1247, 2938, 1179, 3083,...","[1095, 514, 714, 2424, 3776, 3612, 3262, 1333,...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
76018,6040,"[1097, 2076, 3224, 1247, 899, 1103, 2303, 3547...","[926, 720, 1207, 126, 280, 1228, 1632, 647, 12...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."


In [54]:
natural_language_dataset

,user_id,history,candidates,ground_truth
0,5,"Movie 50: The Usual Suspects (Genres: Drama, C...",Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...
1,5,Movie 162: Crumb (Genres: Documentary; Languag...,"Movie 1: Toy Story (Genres: Animation, Comedy,...",Movie 32: Twelve Monkeys (Genres: Science Fict...
2,5,"Movie 176: Living in Oblivion (Genres: Drama, ...","Movie 31: Dangerous Minds (Genres: Drama, Crim...",Movie 32: Twelve Monkeys (Genres: Science Fict...
3,5,Movie 162: Crumb (Genres: Documentary; Languag...,Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...
4,5,Movie 162: Crumb (Genres: Documentary; Languag...,Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...
5,5,Movie 348: Bullets Over Broadway (Genres: Acti...,"Movie 714: Dead Man (Genres: Drama, Fantasy, W...",Movie 32: Twelve Monkeys (Genres: Science Fict...
6,5,"Movie 50: The Usual Suspects (Genres: Drama, C...","Movie 379: Timecop (Genres: Thriller, Science ...",Movie 32: Twelve Monkeys (Genres: Science Fict...
7,5,Movie 29: The City of Lost Children (Genres: F...,Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...
8,5,Movie 29: The City of Lost Children (Genres: F...,Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...
9,5,Movie 29: The City of Lost Children (Genres: F...,Movie 32: Twelve Monkeys (Genres: Science Fict...,Movie 32: Twelve Monkeys (Genres: Science Fict...


In [45]:
len(natural_language_dataset["history"][0])

1488

In [57]:
natural_language_dataset["history"][0]

'Movie 50: The Usual Suspects (Genres: Drama, Crime, Thriller; Language: en; Overview: Verbal Kint tells feds about mythic crime lord Keyser Soze.),Movie 176: Living in Oblivion (Genres: Drama, Comedy; Language: en; Overview: Film about indie filmmaking, set in one day on set.),Movie 515: The Remains of the Day (Genres: Drama, Romance; Language: en; Overview: Butler\'s decorum challenged by love and political tensions post-WWI.),Movie 562: Welcome to the Dollhouse (Genres: Comedy, Drama; Language: en; Overview: Seventh grader faces bullying and neglect in suburban family life.),Movie 1171: Bob Roberts (Genres: Comedy, Drama; Language: en; Overview: Mockumentary explores a Senate candidate\'s media manipulation through music.),Movie 1392: Citizen Ruth (Genres: Drama, Comedy; Language: en; Overview: "Citizen Ruth" satirizes media frenzy over one woman\'s pregnancy.),Movie 1732: The Big Lebowski (Genres: Comedy, Crime; Language: en; Overview: Slacker mistaken for millionaire, faces bizarr

In [55]:
natural_language_dataset.to_csv('dataset/movielens_1M/natural_language_top10_sample.csv', index=False)



Build into DPO dataset

In [50]:
natural_language_dataset = seq_dataset.apply(process_sequence, axis=1)


In [51]:
natural_language_dataset.to_csv('dataset/movielens_1M/natural_language_top10.csv', index=False)


In [4]:
import pandas as pd

# 读取CSV文件
train_set_path = 'dataset/movielens_1M/test_set.csv'
train_set = pd.read_csv(train_set_path)

# 检查是否包含指定的movieId
movie_id_to_check = 3358
if movie_id_to_check in train_set['movieId'].values:
    print(f"movieId {movie_id_to_check} 存在于数据集中。")
else:
    print(f"movieId {movie_id_to_check} 不存在于数据集中。")


movieId 3358 存在于数据集中。


In [1]:
import pandas as pd
recommendations_df = pd.read_csv("/Users/lavander/cpan/h411711/dataset/movielens_1M/test_set.csv")

recommendations_df = recommendations_df.sort_values(by="userId").reset_index(drop=True)

# 保存排序后的推荐结果
recommendations_df.to_csv("/Users/lavander/cpan/h411711/dataset/movielens_1M/test_set_sorted.csv", index=False)

print("推荐结果已按 user_id 排序并保存为 'cf/item_cf_recommendations_sorted.csv'。")

推荐结果已按 user_id 排序并保存为 'cf/item_cf_recommendations_sorted.csv'。


In [2]:
import pandas as pd
recommendations_df = pd.read_csv("/Users/lavander/cpan/h411711/dataset/movielens_1M/train_set.csv")

recommendations_df = recommendations_df.sort_values(by="userId").reset_index(drop=True)

# 保存排序后的推荐结果
recommendations_df.to_csv("/Users/lavander/cpan/h411711/dataset/movielens_1M/train_set_sorted.csv", index=False)

print("推荐结果已按 user_id 排序并保存为 'cf/item_cf_recommendations_sorted.csv'。")

推荐结果已按 user_id 排序并保存为 'cf/item_cf_recommendations_sorted.csv'。


In [5]:
import pandas as pd
recommendations_df = pd.read_csv("/Users/lavander/cpan/h411711/RippleNet-PyTorch-baseline/src/results/natural_language.csv")
recommendations_df

,user_id,history,candidates,ground_truth
0,5,"Movie 506: Orlando (Genres: Romance, Drama, Fa...",NaN,NaN
1,8,"Movie 58: The Postman (Genres: Comedy, Drama, ...",NaN,NaN
2,10,Movie 750: Dr. Strangelove or: How I Learned t...,NaN,NaN
3,15,Movie 104: Happy Gilmore (Genres: Comedy; Lang...,NaN,NaN
4,17,Movie 32: Twelve Monkeys (Genres: Science Fict...,NaN,NaN
...,...,...,...,...
995,2445,"Movie 434: Cliffhanger (Genres: Action, Advent...",NaN,NaN
996,2446,Movie 1132: Manon of the Spring (Genres: Drama...,NaN,NaN
997,2451,"Movie 480: Jurassic Park (Genres: Adventure, S...",NaN,NaN
998,2453,"Movie 58: The Postman (Genres: Comedy, Drama, ...",NaN,NaN
